# CREACION DEL DATAMART ANALITICO

## SET UP 

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

%config IPCompleter.greedy = True

pd.options.display.float_format = '{:15.2f}'.format

## CARGA DE DATOS 

In [7]:
import sqlalchemy as sa
con = sa.create_engine('sqlite:///../Datos/ecommerce.db')

In [8]:
from sqlalchemy import inspect 

insp = inspect(con)
insp.get_table_names()

['2019-Dec', '2019-Nov', '2019-Oct', '2020-Feb', '2020-Jan']

In [9]:
oct = pd.read_sql('2019-Dec',con)
nov = pd.read_sql('2019-Nov',con)
dic = pd.read_sql('2019-Oct',con)
ene = pd.read_sql('2020-Feb',con)
feb = pd.read_sql('2020-Jan',con)

## INTEGRACION DE DATOS 

In [10]:
df = pd.concat([oct, nov, dic, ene, feb], axis = 0)
df

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,22,2019-12-01 00:01:02 UTC,view,5706778,1487580005268456287,None,beautix,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de
1,34,2019-12-01 00:01:50 UTC,view,5795703,1487580005268456287,None,None,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f
2,75,2019-12-01 00:03:25 UTC,view,5771114,1487580005511725929,None,None,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890
3,90,2019-12-01 00:03:48 UTC,view,5884577,1487580006350586771,appliances.environment.vacuum,max,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481
4,91,2019-12-01 00:03:49 UTC,remove_from_cart,5899853,2115334439910245200,None,None,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce
...,...,...,...,...,...,...,...,...,...,...
443219,4264714,2020-01-31 23:58:26 UTC,purchase,5730206,1487580005092295511,None,None,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443220,4264715,2020-01-31 23:58:26 UTC,purchase,5861342,1487580010645553231,None,None,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443221,4264716,2020-01-31 23:58:26 UTC,purchase,5867185,1487580007910867929,None,staleks,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443222,4264717,2020-01-31 23:58:26 UTC,purchase,5877766,1605161575889502297,None,None,4.76,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e


## CALIDAD DE DATOS 

### Tipos de variables

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095076 entries, 0 to 443223
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   index          int64  
 1   event_time     object 
 2   event_type     object 
 3   product_id     int64  
 4   category_id    int64  
 5   category_code  object 
 6   brand          object 
 7   price          float64
 8   user_id        int64  
 9   user_session   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 175.8+ MB


Eliminamos la columna index.

In [12]:
df.drop(columns = 'index', inplace = True)

Análisis y corrección de tipos.

Pasamos event_time a datetime.

In [1]:
#forma tradicional
#df.event_time = pd.to_datetime(df.event_time)

In [16]:
#forma avanzada creando una función
#a esta función hay que pasarle la variable fecha y el formato en el que está

def datetime_rapido(dt, formato):
    def divide_fecha(fecha):
        division = fecha.split()
        date = division[0]
        time = division[1]
        cadena = date + ' ' + time
        return(cadena)
    
    resultado = pd.to_datetime(dt.apply(lambda x: divide_fecha(x)), format = formato)
    
    return(resultado)

Ejecutamos la función.

In [19]:
formato = '%Y-%m-%d %H:%M:%S'

df.event_time = datetime_rapido(df.event_time, formato)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095076 entries, 0 to 443223
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  object        
 5   brand          object        
 6   price          float64       
 7   user_id        int64         
 8   user_session   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 159.8+ MB


### Nombres de variables

Renombramos las variables a español.

In [20]:
df.columns = ['fecha',
              'evento',
              'producto',
              'categoria',
              'categoria_cod',
              'marca',
              'precio',
              'usuario',
              'sesion']
df

,fecha,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion
0,2019-12-01 00:01:02,view,5706778,1487580005268456287,None,beautix,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de
1,2019-12-01 00:01:50,view,5795703,1487580005268456287,None,None,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f
2,2019-12-01 00:03:25,view,5771114,1487580005511725929,None,None,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890
3,2019-12-01 00:03:48,view,5884577,1487580006350586771,appliances.environment.vacuum,max,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481
4,2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,None,None,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce
...,...,...,...,...,...,...,...,...,...
443219,2020-01-31 23:58:26,purchase,5730206,1487580005092295511,None,None,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443220,2020-01-31 23:58:26,purchase,5861342,1487580010645553231,None,None,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443221,2020-01-31 23:58:26,purchase,5867185,1487580007910867929,None,staleks,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443222,2020-01-31 23:58:26,purchase,5877766,1605161575889502297,None,None,4.76,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e


### Análisis de nulos

In [22]:
df.isna().sum().sort_values(ascending = False)

categoria_cod    2060411
marca             891646
sesion               506
fecha                  0
evento                 0
producto               0
categoria              0
precio                 0
usuario                0
dtype: int64

Conclusiones:

* categoria_cod tiene casi todos los registros a nulo
* marca tiene casi la mitad de los registros a nulo
* hay 506 nulos en sesión

Acciones:

* eliminar las variables categoria_cod y marca
* eliminar los nulos de sesión ya que es una variable relevante

In [24]:
df = df.drop(columns = ['categoria_cod','marca']).dropna()
df

,fecha,evento,producto,categoria,precio,usuario,sesion
0,2019-12-01 00:01:02,view,5706778,1487580005268456287,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de
1,2019-12-01 00:01:50,view,5795703,1487580005268456287,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f
2,2019-12-01 00:03:25,view,5771114,1487580005511725929,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890
3,2019-12-01 00:03:48,view,5884577,1487580006350586771,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481
4,2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce
...,...,...,...,...,...,...,...
443219,2020-01-31 23:58:26,purchase,5730206,1487580005092295511,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443220,2020-01-31 23:58:26,purchase,5861342,1487580010645553231,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443221,2020-01-31 23:58:26,purchase,5867185,1487580007910867929,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443222,2020-01-31 23:58:26,purchase,5877766,1605161575889502297,4.76,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e


### Análisis de las variables numéricas

In [25]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
producto,2094570.00,5487103.56,1300923.90,3752.00,5724652.00,5811665.00,5858353.00,5932595.00
categoria,2094570.00,1553112489392095744.00,167907497920583712.00,1487580004807082752.00,1487580005754995456.00,1487580008246412288.00,1487580013489291520.00,2242903426784559104.00
precio,2094570.00,8.42,19.14,-47.62,2.05,4.00,6.86,327.78
usuario,2094570.00,521077545.56,87553855.76,4661182.00,480613387.00,553341613.00,578406571.00,622087993.00


Vemos negativos en el precio. Vamos a profundizar.

In [28]:
df[df.precio <= 0]

,fecha,evento,producto,categoria,precio,usuario,sesion
1527,2019-12-01 07:04:11,view,5904954,1487580012902088873,0.00,574593006,51cc2f4d-5b06-4358-b2a4-6c2f8d6e70c1
2385,2019-12-01 08:24:20,view,5875183,1487580006300255120,0.00,536105376,9bc57104-976e-4dab-8b59-3c5b99ed5fbf
6383,2019-12-01 12:44:26,view,5909785,1487580006350586771,0.00,399445659,10c0b290-fcd6-467b-b1d6-08ba66c8ddc5
6384,2019-12-01 12:44:40,cart,5909785,1487580006350586771,0.00,399445659,10c0b290-fcd6-467b-b1d6-08ba66c8ddc5
6403,2019-12-01 12:47:37,view,5909785,1487580006350586771,0.00,399445659,10c0b290-fcd6-467b-b1d6-08ba66c8ddc5
...,...,...,...,...,...,...,...
442443,2020-01-31 21:27:54,view,5914498,1783999072332415142,0.00,599666842,d881323d-2911-43bf-9557-6cdf0fd392c7
442461,2020-01-31 21:30:40,view,5924098,1487580013128581300,0.00,553588523,d766e6ad-6c19-41a2-be1c-51c306bae280
442462,2020-01-31 21:30:50,cart,5924098,1487580013128581300,0.00,553588523,d766e6ad-6c19-41a2-be1c-51c306bae280
442696,2020-01-31 21:59:40,remove_from_cart,5858319,1783999072332415142,0.00,599666842,d881323d-2911-43bf-9557-6cdf0fd392c7


Son unos 20000 registros, podríamos eliminarlos.

Pero antes ¿se concentran quizá en algún producto determinado?

In [29]:
df[df.precio <= 0].producto.value_counts().head(10)

5896186    79
5903915    50
5873428    37
5851304    29
5851294    29
5837624    28
5851272    27
5712583    27
5899512    26
5907812    26
Name: producto, dtype: int64

No parece que sea problema de un producto concreto, así que vamos a eliminar todos los registros.

In [30]:
df = df[df.precio > 0]
df

,fecha,evento,producto,categoria,precio,usuario,sesion
0,2019-12-01 00:01:02,view,5706778,1487580005268456287,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de
1,2019-12-01 00:01:50,view,5795703,1487580005268456287,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f
2,2019-12-01 00:03:25,view,5771114,1487580005511725929,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890
3,2019-12-01 00:03:48,view,5884577,1487580006350586771,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481
4,2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce
...,...,...,...,...,...,...,...
443219,2020-01-31 23:58:26,purchase,5730206,1487580005092295511,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443220,2020-01-31 23:58:26,purchase,5861342,1487580010645553231,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443221,2020-01-31 23:58:26,purchase,5867185,1487580007910867929,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
443222,2020-01-31 23:58:26,purchase,5877766,1605161575889502297,4.76,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e


### Análisis de las variables categóricas

In [31]:
df.evento.nunique()

4

In [32]:
df.evento.value_counts()

view                961558
cart                574547
remove_from_cart    410357
purchase            127564
Name: evento, dtype: int64

In [33]:
df.producto.nunique()

45327

In [34]:
df.categoria.nunique()

508

### Índice

Vamos a poner la fecha como el index.

In [36]:
df.set_index('fecha', inplace = True)
df

,evento,producto,categoria,precio,usuario,sesion
fecha,,,,,,
2019-12-01 00:01:02,view,5706778,1487580005268456287,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de
2019-12-01 00:01:50,view,5795703,1487580005268456287,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f
2019-12-01 00:03:25,view,5771114,1487580005511725929,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890
2019-12-01 00:03:48,view,5884577,1487580006350586771,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481
2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce
...,...,...,...,...,...,...
2020-01-31 23:58:26,purchase,5730206,1487580005092295511,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
2020-01-31 23:58:26,purchase,5861342,1487580010645553231,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
2020-01-31 23:58:26,purchase,5867185,1487580007910867929,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e


## TRANSFORMACION DE DATOS

Crear 3 tipos de nuevas variables

* Extraer componentes
* Variables de calendario: Festivos locales de Rusia
* Indicadores exógenos: Días de interés comercial: Black Friday, Cyber Monday, Reyes, San Valentin

### Componentes de la fecha

In [37]:
def componentes_fecha(dataframe):
    date = dataframe.index.date
    año = dataframe.index.year
    mes = dataframe.index.month
    dia = dataframe.index.day
    hora = dataframe.index.hour
    minuto = dataframe.index.minute
    segundo = dataframe.index.second
    
    
    return(pd.DataFrame({'date':date, 'año':año,'mes':mes, 'dia':dia, 'hora':hora, 'minuto':minuto, 'segundo':segundo}))

In [40]:
df = pd.concat([df.reset_index(), componentes_fecha(df)], axis = 1).set_index('fecha')
df

,evento,producto,categoria,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo
fecha,,,,,,,,,,,,,
2019-12-01 00:01:02,view,5706778,1487580005268456287,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de,2019-12-01,2019,12,1,0,1,2
2019-12-01 00:01:50,view,5795703,1487580005268456287,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f,2019-12-01,2019,12,1,0,1,50
2019-12-01 00:03:25,view,5771114,1487580005511725929,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890,2019-12-01,2019,12,1,0,3,25
2019-12-01 00:03:48,view,5884577,1487580006350586771,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481,2019-12-01,2019,12,1,0,3,48
2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce,2019-12-01,2019,12,1,0,3,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31 23:58:26,purchase,5730206,1487580005092295511,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26
2020-01-31 23:58:26,purchase,5861342,1487580010645553231,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26
2020-01-31 23:58:26,purchase,5867185,1487580007910867929,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26


### Variables de calendario: festivos

Para incorporar festivos podemos usar el paquete holidays.

Tiene fiestas de varios países e incluso a nivel comunidades.

Instalacion: conda install -c conda-forge holidays

Importacion: import holidays

Guia rapido y lista de paises:

https://github.com/dr-prodigy/python-holidays

Probamos con España.

In [48]:
import holidays

festivo_es = holidays.ES(years = 2021)

for fecha, fiesta in festivo_es.items():
    print(fecha,fiesta)

2021-01-01 Año nuevo
2021-01-06 Epifanía del Señor
2021-04-01 Jueves Santo
2021-04-02 Viernes Santo
2021-05-01 Día del Trabajador
2021-08-16 Asunción de la Virgen (Trasladado)
2021-10-12 Día de la Hispanidad
2021-11-01 Todos los Santos
2021-12-06 Día de la Constitución Española
2021-12-08 La Inmaculada Concepción
2021-12-25 Navidad


Definimos el objeto festivo_ru ya que este ecommerce es Ruso.

In [49]:
festivo_ru = holidays.RU(years=2020)
festivo_ru

{datetime.date(2020, 1, 1): 'Новогодние каникулы', datetime.date(2020, 1, 2): 'Новогодние каникулы', datetime.date(2020, 1, 3): 'Новогодние каникулы', datetime.date(2020, 1, 4): 'Новогодние каникулы', datetime.date(2020, 1, 5): 'Новогодние каникулы', datetime.date(2020, 1, 6): 'Новогодние каникулы', datetime.date(2020, 1, 8): 'Новогодние каникулы', datetime.date(2020, 1, 7): 'Рождество Христово', datetime.date(2020, 2, 23): 'День защитника Отечества', datetime.date(2020, 3, 8): 'Международный женский день', datetime.date(2020, 5, 1): 'Праздник Весны и Труда', datetime.date(2020, 5, 9): 'День Победы', datetime.date(2020, 6, 12): 'День России', datetime.date(2020, 11, 4): 'День народного единства'}

Incorporar una variable que diga en cada registro si era un día festivo o no.

In [53]:
df['festivo'] = df.date.apply(lambda x: 1 if (x in festivo_ru) else 0)
df

,evento,producto,categoria,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo,festivo
fecha,,,,,,,,,,,,,,
2019-12-01 00:01:02,view,5706778,1487580005268456287,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de,2019-12-01,2019,12,1,0,1,2,0
2019-12-01 00:01:50,view,5795703,1487580005268456287,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f,2019-12-01,2019,12,1,0,1,50,0
2019-12-01 00:03:25,view,5771114,1487580005511725929,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890,2019-12-01,2019,12,1,0,3,25,0
2019-12-01 00:03:48,view,5884577,1487580006350586771,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481,2019-12-01,2019,12,1,0,3,48,0
2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce,2019-12-01,2019,12,1,0,3,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31 23:58:26,purchase,5730206,1487580005092295511,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26,0
2020-01-31 23:58:26,purchase,5861342,1487580010645553231,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26,0
2020-01-31 23:58:26,purchase,5867185,1487580007910867929,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26,0


Comprobamos los festivos.

In [54]:
df[df.festivo == 1].date.value_counts().sort_index()

2019-11-04    16430
2020-01-01     7644
2020-01-02    10776
2020-01-03    10617
2020-01-04    13084
2020-01-05    14554
2020-01-06    10621
2020-01-07    12922
2020-01-08    14004
2020-02-23     9817
Name: date, dtype: int64

### Indicadores exógenos

Vamos a añadir indicadores para Black Friday y San Valentín.

In [55]:
df['black_friday'] = 0
df.loc['2019-11-29','black_friday'] = 1

df['san_valentin'] = 0
df.loc['2019-02-14','san_valentin'] = 1

Comprobamos.

In [56]:
df['black_friday'].value_counts()

0    2051695
1      22331
Name: black_friday, dtype: int64

In [57]:
df['san_valentin'].value_counts()

0    2074026
Name: san_valentin, dtype: int64

In [58]:
df

,evento,producto,categoria,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo,festivo,black_friday,san_valentin
fecha,,,,,,,,,,,,,,,,
2019-12-01 00:01:02,view,5706778,1487580005268456287,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de,2019-12-01,2019,12,1,0,1,2,0,0,0
2019-12-01 00:01:50,view,5795703,1487580005268456287,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f,2019-12-01,2019,12,1,0,1,50,0,0,0
2019-12-01 00:03:25,view,5771114,1487580005511725929,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890,2019-12-01,2019,12,1,0,3,25,0,0,0
2019-12-01 00:03:48,view,5884577,1487580006350586771,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481,2019-12-01,2019,12,1,0,3,48,0,0,0
2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce,2019-12-01,2019,12,1,0,3,49,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31 23:58:26,purchase,5730206,1487580005092295511,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26,0,0,0
2020-01-31 23:58:26,purchase,5861342,1487580010645553231,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26,0,0,0
2020-01-31 23:58:26,purchase,5867185,1487580007910867929,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,2020-01-31,2020,1,31,23,58,26,0,0,0


## TABLON ANALITICO FINAL

Revisamos lo que tenemos.

In [59]:
df.head()

,evento,producto,categoria,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo,festivo,black_friday,san_valentin
fecha,,,,,,,,,,,,,,,,
2019-12-01 00:01:02,view,5706778,1487580005268456287,14.13,564257889,a4248817-8d0d-4dac-96e6-a586563308de,2019-12-01,2019,12,1,0,1,2,0,0,0
2019-12-01 00:01:50,view,5795703,1487580005268456287,10.95,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f,2019-12-01,2019,12,1,0,1,50,0,0,0
2019-12-01 00:03:25,view,5771114,1487580005511725929,4.27,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890,2019-12-01,2019,12,1,0,3,25,0,0,0
2019-12-01 00:03:48,view,5884577,1487580006350586771,47.46,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481,2019-12-01,2019,12,1,0,3,48,0,0,0
2019-12-01 00:03:49,remove_from_cart,5899853,2115334439910245200,4.46,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce,2019-12-01,2019,12,1,0,3,49,0,0,0


Reorganizamos

In [60]:
variables = df.columns.to_list()
variables

['evento',
 'producto',
 'categoria',
 'precio',
 'usuario',
 'sesion',
 'date',
 'año',
 'mes',
 'dia',
 'hora',
 'minuto',
 'segundo',
 'festivo',
 'black_friday',
 'san_valentin']

In [61]:
orden = ['usuario',
         'sesion',
         'categoria',
         'evento',
         'producto',
         'precio']
orden

['usuario', 'sesion', 'categoria', 'evento', 'producto', 'precio']

In [64]:
resto = [nombre for nombre in variables if nombre not in orden]
resto

['date',
 'año',
 'mes',
 'dia',
 'hora',
 'minuto',
 'segundo',
 'festivo',
 'black_friday',
 'san_valentin']

In [66]:
df = df[orden + resto]
df

,usuario,sesion,categoria,evento,producto,precio,date,año,mes,dia,hora,minuto,segundo,festivo,black_friday,san_valentin
fecha,,,,,,,,,,,,,,,,
2019-12-01 00:01:02,564257889,a4248817-8d0d-4dac-96e6-a586563308de,1487580005268456287,view,5706778,14.13,2019-12-01,2019,12,1,0,1,2,0,0,0
2019-12-01 00:01:50,556579890,1d94a188-7fa5-4c52-a9fc-99abf8b4838f,1487580005268456287,view,5795703,10.95,2019-12-01,2019,12,1,0,1,50,0,0,0
2019-12-01 00:03:25,509636330,4e930dc6-7cf6-419f-8297-1edd6cd20890,1487580005511725929,view,5771114,4.27,2019-12-01,2019,12,1,0,3,25,0,0,0
2019-12-01 00:03:48,499259978,4ee80bbb-17cb-447d-9188-9aa306db3481,1487580006350586771,view,5884577,47.46,2019-12-01,2019,12,1,0,3,48,0,0,0
2019-12-01 00:03:49,579950721,f39acf4b-11d5-479d-84a7-04f5bc6255ce,2115334439910245200,remove_from_cart,5899853,4.46,2019-12-01,2019,12,1,0,3,49,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31 23:58:26,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,1487580005092295511,purchase,5730206,10.32,2020-01-31,2020,1,31,23,58,26,0,0,0
2020-01-31 23:58:26,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,1487580010645553231,purchase,5861342,1.90,2020-01-31,2020,1,31,23,58,26,0,0,0
2020-01-31 23:58:26,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e,1487580007910867929,purchase,5867185,11.43,2020-01-31,2020,1,31,23,58,26,0,0,0


Guardamos como pickle preserbar los metadatos.

In [67]:
df.to_pickle('../Datos/tablon_analitico.pickle')